In [9]:
from configparser import ConfigParser
from sqlalchemy import create_engine
import pandas as pd
#import pandas_profiling
from ua_parser import user_agent_parser
from geolite2 import geolite2
import geopandas as gpd
import pycountry
import re
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline
sns.set(color_codes=True)
sns.set_palette('autumn')
from pandas.api.types import is_numeric_dtype


def read_db_config(filename='config-dump.ini', section='mysql'):
    """ Read database configuration file and return a dictionary object
    :param filename: name of the configuration file
    :param section: section of database configuration
    :return: a dictionary of database parameters
    """
    # create parser and read ini configuration file
    parser = ConfigParser()
    parser.read(filename)
 
    # get section, default to mysql
    db = {}
    if parser.has_section(section):
        items = parser.items(section)
        for item in items:
            db[item[0]] = item[1]
    else:
        raise Exception('{0} not found in the {1} file'.format(section, filename))
 
    return db

dbconfig = read_db_config()
CONN = 'mysql+mysqlconnector://'+dbconfig['user']+':'+dbconfig['pwd']+'@'+dbconfig['host']+':'+dbconfig['port'] + '/'+dbconfig['dbname']
engine = create_engine(CONN)

# Supress scientific notation 
pd.set_option('display.float_format', lambda x: '%.3f' % x)

In [12]:
P_SAMPLE = 'select * from VALID_PARTICIPANTS'
p_sample = pd.read_sql_query(P_SAMPLE, engine)
p_sample.fillna(0, inplace=True)
p_sample.head()

PARTICIPANT_ID      IP_ADDRESS  \
0             629   110.54.203.10   
1             632  115.164.40.232   
2             669   122.8.244.180   
3             673   37.219.99.214   
4             674   118.93.226.60   

                                             BROWSER BROWSER_LANGUAGE  DEVICE  \
0  Mozilla/5.0 (Linux; Android 5.0.2; vivo Y51 Bu...            en-US  mobile   
1  Mozilla/5.0 (Linux; Android 7.0; vivo 1713 Bui...            en-US  mobile   
2  Mozilla/5.0 (Linux; Android 6.0; LG-V495 Build...   en-US,en;q=0.8  mobile   
3  Mozilla/5.0 (Linux; Android 6.0; HTC One X9 du...   en-US,en;q=0.9  mobile   
4  Mozilla/5.0 (iPhone; CPU iPhone OS 11_4_1 like...            en-us  mobile   

   SCREEN_W  SCREEN_H  AGE  GENDER  HAS_TAKEN_TYPING_COURSE     ...       \
0       360       640   20    male                        0     ...        
1       360       640   24    male                        0     ...        
2       601       962   16  female                        0     ...        
3       360       640   27  female                        0     ...        
4       375       667   24  female                        0     ...        

  TYPE_TEST_LANG  P_KPD   P_IKI P_BSP P_ECPC P_UILEN P_KSPC ITE_SWYPE  \
0    sometimes 21.981 188.929 3.467  0.084  42.733  1.163     0.034   
1      usually 14.406 199.738 1.000  0.024  47.133  1.323     0.061   
2      usually 22.416 168.124 0.000  0.000  38.200  1.127     0.116   
3      usually 15.132 120.704 0.000  0.000  41.667  1.061     0.017   
4       always 21.390 137.334 1.600  0.047  38.467  1.011     0.119   

  ITE_PREDICT ITE_AUTOCORR  
0       0.034        0.000  
1       0.157        0.013  
2       0.037        0.046  
3       0.186        0.187  
4       0.246        0.007  

[5 rows x 29 columns]

In [13]:
columns = ['PARTICIPANT_ID', 'WPM', 'ERROR_RATE', 'USING_FEATURES', 
           'P_KPD', 'P_IKI', 'P_BSP', 'P_ECPC', 'P_UILEN', 'P_KSPC', 
           'ITE_SWYPE', 'ITE_PREDICT', 'ITE_AUTOCORR']

p_sample = p_sample[columns]
p_sample['REP_SWYPE'] = p_sample.USING_FEATURES.apply(lambda x: 1 if x.find('swipe') != -1 else 0)

# Find participants whi have not reported using Swype, but ITE_SWYPE != 0. For them assignt ITE_PRED += ITE_SWYPE
reassign_indxs = p_sample[(p_sample['REP_SWYPE']==0) & (p_sample['ITE_SWYPE'] !=0)].index
p_sample.loc[reassign_indxs, 'ITE_PREDICT'] = p_sample.loc[reassign_indxs, 'ITE_PREDICT'] + p_sample.loc[reassign_indxs, 'ITE_SWYPE']
p_sample.loc[reassign_indxs, 'ITE_SWYPE'] = 0


In [14]:
TS_SAMPLE = 'select * from FIRST_15_TEST_SECTIONS'
ts_sample = pd.read_sql_query(TS_SAMPLE, engine)
ts_sample.fillna(0, inplace=True)
ts_sample.head()

TEST_SECTION_ID  SENTENCE_ID  PARTICIPANT_ID  \
0             2512         1269             629   
1             2513         1365             629   
2             2515          401             629   
3             2516          349             629   
4             2520          448             629   

                                          USER_INPUT  INPUT_TIME  \
0              We have lost not just a great friend.       10886   
1                           I do not want to repeat.        6497   
2  Rookie Kevin Swayne will move into the startin...       18116   
3           Theresa, I changed the CIG fuel rate to.       13687   
4                      Weinke could have a big game.        9766   

   EDIT_DISTANCE  ERROR_RATE    WPM  INPUT_LENGTH  ERROR_LEN      ...       \
0              0       0.000 39.684            36         37      ...        
1              0       0.000 42.481            23         24      ...        
2              0       0.000 35.769            54         55      ...        
3              0       0.000 34.193            39         40      ...        
4              0       0.000 34.405            28         29      ...        

   TS_UILEN  TS_KS TS_KSPC  N_SWYP  N_PRED  N_AUTO  PR_SWYP  PR_PRED  PR_AUTO  \
0        37     47   1.270   1.000   1.000   0.000    0.125    0.125    0.000   
1        24     24   1.000   1.000   1.000   0.000    0.167    0.167    0.000   
2        55     59   1.073   0.000   0.000   0.000    0.000    0.000    0.000   
3        40     42   1.050   0.000   0.000   0.000    0.000    0.000    0.000   
4        29     39   1.345   0.000   0.000   0.000    0.000    0.000    0.000   

   TS_NUM_WORDS  
0         8.000  
1         6.000  
2         9.000  
3         8.000  
4         6.000  

[5 rows x 27 columns]

In [15]:
participants = pd.unique(p_sample.PARTICIPANT_ID)
sentences = pd.unique(ts_sample.SENTENCE_ID)

In [16]:
p_sample.tail()

PARTICIPANT_ID    WPM  ERROR_RATE      USING_FEATURES  P_KPD  P_IKI  \
37601          274107 61.682       1.802              ["no"]  5.144 33.553   
37602          274109 31.198       0.638  ["autocorrection"] 11.781 32.085   
37603          274116 42.801       2.326              ["no"]  9.255 35.059   
37604          274128 47.055       0.728              ["no"]  3.471 48.304   
37605          274142 39.407       1.101              ["no"]  4.750 50.689   

       P_BSP  P_ECPC  P_UILEN  P_KSPC  ITE_SWYPE  ITE_PREDICT  ITE_AUTOCORR  \
37601  0.000   0.000   36.733   1.273      0.000        0.000         0.049   
37602  0.000   0.000   41.733   1.554      0.000        0.051         0.052   
37603  0.000   0.000   37.000   1.324      0.000        0.162         0.031   
37604  1.400   0.032   45.600   1.062      0.000        0.000         0.000   
37605  0.000   0.000   42.267   1.141      0.000        0.185         0.037   

       REP_SWYPE  
37601          0  
37602          0  
37603          0  
37604          0  
37605          0

In [17]:
S_SAMPLE = 'select * from SENTENCES where SENTENCE_ID in (select distinct SENTENCE_ID from FIRST_15_TEST_SECTIONS )'
s_sample = pd.read_sql_query(S_SAMPLE, engine)
s_sample.fillna(0, inplace=True)
s_sample.head()

SENTENCE_ID                                           SENTENCE  SOURCE
0            1                         Need before board meeting.  mobile
1            2                           Please set something up.  mobile
2            3                          Is Cassandra there today?  mobile
3            4           If so he probably will have to pay back.  mobile
4            5  Surely we can cover this from within Virgo's t...  mobile

In [18]:
KS_SAMPLE = 'select * from KEYSTROKES where KEYCODE != 229 and TEST_SECTION_ID in (select TEST_SECTION_ID from FIRST_15_TEST_SECTIONS  )'
ks_sample = pd.read_sql_query(KS_SAMPLE, engine)
ks_sample.fillna(0, inplace=True)
ks_sample.head()

KEYSTROKE_ID     PRESS_TIME   RELEASE_TIME LETTER  TEST_SECTION_ID  \
0        107775  1537443061164  1537443061184   BKSP             2512   
1        107776  1537443061339  1537443061367   BKSP             2512   
2        107777  1537443061463  1537443061481   BKSP             2512   
3        107778  1537443061618  1537443061646   BKSP             2512   
4        107779  1537443061780  1537443061824   BKSP             2512   

   KEYCODE     IKI  
0        8 447.000  
1        8  88.000  
2        8 124.000  
3        8 155.000  
4        8 162.000

In [19]:
len(participants)

37606

In [ ]:
import xml.etree.cElementTree as ET
from xml.sax.saxutils import escape, unescape


for p in range(0, len(participants)):
    
    p_id = participants[p]   
    
    rec = p_sample.loc[p_sample[p_sample.PARTICIPANT_ID == p_id].index[0]]

    if (rec.ITE_SWYPE + rec.ITE_PREDICT + rec.ITE_AUTOCORR) == 0:
        method = 'norm'
    elif (rec.ITE_SWYPE != 0 and rec.ITE_PREDICT == 0 and rec.ITE_AUTOCORR == 0):
        method = 'swype'
    elif (rec.ITE_SWYPE == 0 and rec.ITE_PREDICT != 0 and rec.ITE_AUTOCORR == 0):
        method = 'pred'
    elif (rec.ITE_SWYPE == 0 and rec.ITE_PREDICT == 0 and rec.ITE_AUTOCORR != 0):
        method = 'auto'
    else:
        method = 'mix'

    ts_ids = pd.unique(ts_sample[ts_sample.PARTICIPANT_ID==p_id].TEST_SECTION_ID)
    ks_ids = ks_sample[ks_sample.TEST_SECTION_ID == ts_ids[0]].KEYSTROKE_ID.values

    if len(ks_ids > 0):
        start_time = ks_sample[ks_sample.KEYSTROKE_ID==ks_ids[0]]['PRESS_TIME'].values[0]*10000    
        root = ET.Element("TextTest", trials=str(len(ts_ids)), ticks=str(start_time))

        for i in range(0, len(ts_ids)):
            #valid=False
            ts_id = ts_ids[i]
            s_id = ts_sample[ts_sample.TEST_SECTION_ID==ts_id].SENTENCE_ID.values[0]
            sentence = s_sample[s_sample.SENTENCE_ID==s_id].SENTENCE.values[0]
            ks_ids = ks_sample[ks_sample.TEST_SECTION_ID == ts_id].KEYSTROKE_ID.values
            if len(ks_ids>0):
                ks = ks_sample[ks_sample.KEYSTROKE_ID==ks_ids[-1]]['LETTER'].values[0].lower()
            if ks!= ' ':
                doc = ET.SubElement(root, "Trial", testing='true', number=str(i+1), entries=str(len(ks_ids)))
            else:
                doc = ET.SubElement(root, "Trial", testing='true', number=str(i+1), entries=str(len(ks_ids)-1))

            #if len(sentence.strip())<= len(ks_ids):
            #valid=True
            ET.SubElement(doc, "Presented").text=sentence
            for j in range(0, len(ks_ids)):
                ks_id = ks_ids[j]
                ks = ks_sample[ks_sample.KEYSTROKE_ID==ks_id]['LETTER'].values[0].lower()
                if j==len(ks_ids)-1 and ks==' ':
                    ''
                else:
                    if ks=='bksp':
                        ks = '&#8;'

                    if ks=='&#8;' or len(ks)==1:
                        time = ks_sample[ks_sample.KEYSTROKE_ID==ks_id]['PRESS_TIME'].values[0]*10000
                        ET.SubElement(doc, "Entry", char=str(ks), ticks=str(time))

            ET.SubElement(doc, "Transcribed").text=ts_sample[ts_sample.TEST_SECTION_ID==ts_id].USER_INPUT.values[0]

        #if valid:
        tree = ET.ElementTree(root)
        tree.write("xml/"+method+"/participant_"+str(p_id)+".xml", encoding="utf-8")


In [ ]:
p_sample